# Import Bibliotheken

In [67]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt

### Package Versionen

In [68]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [69]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [70]:
comment_df = pd.read_excel('data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,Super nettes Personal gutes Frühstück mit groß...,"Personal, Shop, Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,Sehr höfliche Bedienstete Translated by Google...,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,Immer gerne dort nettes und zuvorkommendes Per...,"Personal, Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,Nettes freundliches Personal Translated by Goo...,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,Normale Tankstelle die aber in der Tank App ni...,"Pricing, Digital Fueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,Dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl, Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,Sehr gut Günstig guter shop und einige andere ...,"Pricing, Shop, Kraftstoffauswahl, Waschanlage,..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,Öffnungszeiten Mo So 06 00 22 00 Uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal, Kraftstoffauswahl"


In [71]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [72]:
# Länge der Test- und Traningsdaten bestimmen
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [73]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

In [74]:
next(iter(train_set))

('Preiswert tanken Translated by Google Fill up inexpensively ', 'Pricing')

## Dataloader

### Tokenisierung

In [75]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

In [76]:
# Beispiel für Anwendung des tokenizer
sentence = "This isn't a very long example."
tokenizer(sentence)

['This', "isn't", 'a', 'very', 'long', 'example', '.']

### Vokabular erstellen

In [77]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [78]:
len(train_set)

2798

In [79]:
tokens = yield_tokens(train_set)

In [80]:
next(iter(tokens))

['Preiswert',
 'tanken',
 'Translated',
 'by',
 'Google',
 'Fill',
 'up',
 'inexpensively']

In [81]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [82]:
len(vocab)

3278

In [83]:
vocab(['Hallo', 'sehr', 'Personal'])

[540, 8, 3]

In [84]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [85]:
onehot = torch.zeros(1,len(vocab))

In [86]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[68, 53, 66]

In [87]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [88]:
import torch
import torch.nn

In [89]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,Super nettes Personal gutes Frühstück mit groß...,"Personal, Shop, Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,Sehr höfliche Bedienstete Translated by Google...,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,Immer gerne dort nettes und zuvorkommendes Per...,"Personal, Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,Nettes freundliches Personal Translated by Goo...,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,Normale Tankstelle die aber in der Tank App ni...,"Pricing, Digital Fueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,Dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl, Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,Sehr gut Günstig guter shop und einige andere ...,"Pricing, Shop, Kraftstoffauswahl, Waschanlage,..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,Öffnungszeiten Mo So 06 00 22 00 Uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal, Kraftstoffauswahl"


#### Kategorien splitten

In [90]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', ' Shop', ' Bistro'],
 ['Personal'],
 ['Personal', ' Waschanlage'],
 ['Personal'],
 ['Pricing', ' Digital Fueling'],
 ['Erscheinungsbild', ' Personal'],
 ['Personal', ' Pricing'],
 ['Personal', ' Pricing', ' Kraftstoffauswahl'],
 ['Personal', ' Öffnungszeiten'],
 ['Personal', ' SB-Waschboxen', ' Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', ' Personal'],
 ['Personal', ' Waschanlage'],
 ['Personal', ' Pricing'],
 ['Personal', ' Pricing'],
 ['Bistro'],
 ['Personal', ' Pricing', ' Shop', ' Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', ' Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', ' Shop'],
 ['Erscheinungsbild'],
 ['Personal', ' Bistro'],
 ['Erscheinungsbild', ' Nacht-/Tankautomat', ' Waschanlage', ' Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', ' Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', ' Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', ' Bistro'],
 ['Erscheinung

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [91]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 ' Shop',
 ' Bistro',
 'Personal',
 'Personal',
 ' Waschanlage',
 'Personal',
 'Pricing',
 ' Digital Fueling',
 'Erscheinungsbild',
 ' Personal',
 'Personal',
 ' Pricing',
 'Personal',
 ' Pricing',
 ' Kraftstoffauswahl',
 'Personal',
 ' Öffnungszeiten',
 'Personal',
 ' SB-Waschboxen',
 ' Waschanlage',
 'Shop',
 'Erscheinungsbild',
 ' Personal',
 'Personal',
 ' Waschanlage',
 'Personal',
 ' Pricing',
 'Personal',
 ' Pricing',
 'Bistro',
 'Personal',
 ' Pricing',
 ' Shop',
 ' Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 ' Pricing',
 'Personal',
 'Personal',
 'Personal',
 ' Shop',
 'Erscheinungsbild',
 'Personal',
 ' Bistro',
 'Erscheinungsbild',
 ' Nacht-/Tankautomat',
 ' Waschanlage',
 ' Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 ' Personal',
 'Personal',
 'Bistro',
 'Pricing',
 ' Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 ' Bistro',
 'Erscheinungsbild',
 ' Personal',
 ' Sanitär',
 ' Bistro',
 

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [92]:
flat_set = set(flat_list)
flat_set

{' AdBlue',
 ' Bistro',
 ' Digital Fueling',
 ' E-Mobilität',
 ' Kartenakzeptanzen',
 ' Kraftstoffauswahl',
 ' Luft',
 ' Nacht-/Tankautomat',
 ' Paketservice',
 ' Personal',
 ' Pricing',
 ' SB-Waschboxen',
 ' Sanitär',
 ' Shop',
 ' Staubsauger',
 ' Tankpool',
 ' Verkehrsanbindung',
 ' WLAN',
 ' Waschanlage',
 ' Werkstatt',
 ' Zapfsäulen',
 ' Öffnungszeiten',
 'AdBlue',
 'Bistro',
 'Digital Fueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [93]:
len(flat_set)

46

#### Dictionary erstellen

In [94]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'AdBlue': 0,
 'WLAN': 1,
 ' WLAN': 2,
 ' Paketservice': 3,
 'SB-Waschboxen': 4,
 ' Kraftstoffauswahl': 5,
 'Sanitär': 6,
 ' Personal': 7,
 'Zapfsäulen': 8,
 ' Verkehrsanbindung': 9,
 ' Bistro': 10,
 'Personal': 11,
 ' E-Mobilität': 12,
 ' AdBlue': 13,
 'Bistro': 14,
 'Pricing': 15,
 ' Nacht-/Tankautomat': 16,
 ' Luft': 17,
 'Kartenakzeptanzen': 18,
 ' Kartenakzeptanzen': 19,
 ' Öffnungszeiten': 20,
 'Digital Fueling': 21,
 'E-Mobilität': 22,
 'Öffnungszeiten': 23,
 ' Digital Fueling': 24,
 'Nacht-/Tankautomat': 25,
 'Luft': 26,
 'Kraftstoffauswahl': 27,
 'E-Fuels': 28,
 ' Zapfsäulen': 29,
 'Tankpool': 30,
 ' Werkstatt': 31,
 'Erscheinungsbild': 32,
 'Verkehrsanbindung': 33,
 'Shop': 34,
 ' Staubsauger': 35,
 ' Tankpool': 36,
 ' SB-Waschboxen': 37,
 'Werkstatt': 38,
 'Waschanlage': 39,
 ' Shop': 40,
 ' Waschanlage': 41,
 ' Sanitär': 42,
 'Staubsauger': 43,
 'Paketservice': 44,
 ' Pricing': 45}

#### Tensor

In [95]:
doc2 = label_list_export[2]
doc2

['Personal', ' Waschanlage']

In [101]:
pos_vals = [label_dict[val] for val in doc2]
pos_vals

[11, 41]

In [102]:
labels = torch.LongTensor(pos_vals)

In [105]:
y_onehot = nn.functional.one_hot(labels, num_classes=5)

RuntimeError: Class values must be smaller than num_classes.

In [ ]:
y_onehot = y_onehot.sum(dim=0).float()

In [ ]:
# labels = torch.tensor([1,5,8,0])
# labels = labels.unsqueeze(0)
# target = torch.zeros(labels.size(0), 24).scatter_(1, labels, 1.)
# print(target)

tensor([[1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]])


### Erstellung Dataloader

In [ ]:
# label_pipeline = lambda x: int(x) -1

In [ ]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        label_list.append(label_pipeline(_label))
        
        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.tensor(label_list, dtype=torch.int64)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)
    
    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [ ]:
train_loader = DataLoader(
 train_set, batch_size=64,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)

## Architektur und Training

### Architektur

In [ ]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
vocab_size = len(vocab)
num_class = 5
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [ ]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.CrossEntropyLoss()

## Initialisierung Optimizer
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate, momentum=0.9)

## Definition der Epochen
num_epochs = 20

In [ ]:
train_accuracy = torchmetrics.Accuracy().to(device)

loss_hist = {}
accuracy_hist = {}

### Training funktioniert nicht !!!

In [ ]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)
        # Der Fehler wird berechnet
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label)
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().item()
    train_accuracy.reset()

ValueError: invalid literal for int() with base 10: 'Personal'

### Auswertung Fehlerentwicklung

In [ ]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [ ]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
)
# glue('train-loss-team', train_chart,display=True)

### Auswertung Genauigkeit

In [ ]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [ ]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
)

## Evaluierung

In [ ]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=3247, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=5, bias=True)
)

### Gesamtevaluierung

In [ ]:
test_loader = DataLoader(test_set, batch_size= 64, shuffle=True,collate_fn=collate_batch)

In [ ]:
valid_accuracy = torchmetrics.Accuracy().to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label)
    total_valid_accuracy = valid_accuracy.compute()

ValueError: invalid literal for int() with base 10: 'Shop, Bistro'

In [ ]:
total_valid_accuracy

NameError: name 'total_valid_accuracy' is not defined